# Job Sieve
---

Development documentation for the screen scraping of multiple job boards for useful statistics, filtered high-value prospects, and correlative analytics.

---

## <a name="toc"></a> Table of Contents
1. [Proof of Concept](#poc)



## <a name="poc"></a> [Proof of Concept](#toc)

...



In [ ]:
# -------------------- LOAD DEPENDENCIES -------------------- #

# Environment hard reset
%reset -f

# Standard math and data libraries
import numpy as np
import pandas as pd

# Plotting libraries
import matplotlib.pyplot as plt
%matplotlib inline

# Libraries for scraping
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import lxml.html as lh

# Date time for date operations
import datetime

# Levenshtein fuzzy comparisons
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

# Import string cleaning functions
import re

# Flask support
from flask import request, jsonify

# Configure paths
from pathlib import Path
data_path = Path('Datasets')
